# Import Libraries

In [1]:
import os 
import numpy as np
import tensorflow as tf
from tensorflow import keras as kr
from glob import glob
import matplotlib 
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
from keras.preprocessing.image import save_img

from tensorflow.keras.utils import to_categorical # calculate loss function with multiple classes


from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.constraints import max_norm #
import tensorflow.keras.callbacks

Using TensorFlow backend.


In [2]:
tf.__version__

'2.2.0'

In [3]:
import os
print(os.listdir())

['dataRename.ipynb', 'DataRescale.ipynb', 'Setup_picture', 'testdata', 'Ausarbeitung', 'DATA', 'RunTB.py', 'sequence_classifier.drawio', 'audio', 'component_diagramm.drawio', 'Roboto', 'plot_confusion_matrix.py', 'Evaluation.ipynb', '.ipynb_checkpoints', 'Confusion_matrix.ipynb', 'Training-Primary.ipynb', 'TimeMeasurement.ipynb', 'statemachine.drawio', 'Filter_Pictures.ipynb', 'TODO', 'README', 'Recording', 'sequence_timer.drawio', '__pycache__', 'convertToTFLITE.ipynb', 'Training-Copy.ipynb']


# Data Input Directory

In [4]:
DIMENSION = (128, 128)
DIR_PICS = "pics/OneSortOfClothes/OneSort/DayNight/"
train_data_dir = DIR_PICS + "/data/"
val_data_dir = DIR_PICS + "/val_data/"
nb_train_samples = len(glob('%s/*/*.png' % train_data_dir))
nb_classes = len(glob('%s/*/' % train_data_dir))


In [5]:
print(nb_classes)
print(nb_train_samples)
#print(glob('%s/*/' % train_data_dir))

#grey UP night

5
1773


# Data Augmentation Setup

In [6]:
width_shift = int(DIMENSION[0]*0.3)
height_shift = 0#int(DIMENSION[1]*0.2)
flip=True
rotation_angle= 10# final rotations will be in the range [-rotation_angle, +rotation_angle]
zoom = [0.8, 1.2] #  [lower, upper]
shear = 5 # in degrees
brightness = [0.1, 1.5]


train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=width_shift,
    height_shift_range=height_shift,
    rotation_range=rotation_angle,
    zoom_range=zoom,
    shear_range=shear,
    horizontal_flip=flip,
    brightness_range=brightness)

val_datagen = ImageDataGenerator(rescale=1./255)

# Seperating into batches and shuffling Data (Used for Training)

In [7]:
batchsize = 32
color_mode = 'rgb'

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=DIMENSION,
        batch_size=batchsize, #Divide the images into baches - 5 means into 5 batches
        color_mode=color_mode,
        class_mode='categorical',
        shuffle=True,
        seed=42)

val_generator = val_datagen.flow_from_directory(
        val_data_dir,
        target_size=DIMENSION,
        batch_size=1,
        color_mode=color_mode,
        class_mode='categorical',
        shuffle=True,
        seed=42)

Found 1773 images belonging to 5 classes.
Found 212 images belonging to 5 classes.


In [8]:
print(train_generator.class_indices)

##Just for later use
#classes = np.array(["drinking", "noperson", "crooked", "straight", "phone", "ytvideos"])
#classes[0]

{'crooked': 0, 'drinking': 1, 'noperson': 2, 'phone': 3, 'straight': 4}


# NN 2

In [26]:
if color_mode == 'grayscale':
    DIMENSION_NN = DIMENSION + (1,)
elif color_mode == 'rgb':
    DIMENSION_NN = DIMENSION + (3,)

print(DIMENSION_NN)

model = Sequential()

# conv block 1
model.add(Conv2D(32, kernel_size=(2,2), activation="relu",input_shape=DIMENSION_NN,kernel_constraint=max_norm(3.), name="Conv2D_1")) # single number = biggest difference between two input vectors
#model.add(Conv2D(32, kernel_size=(2,2), activation="relu",kernel_constraint=max_norm(3.)))
model.add(MaxPooling2D(pool_size= (2,2), name="MaxPooling2D_1"))

#conv block 2
model.add(Conv2D(64, kernel_size=(4,4),activation="relu", kernel_constraint=max_norm(3.), name="Conv2D_2"))
#model.add(Conv2D(64, kernel_size=(3,3),activation="relu", kernel_constraint=max_norm(3.)))
model.add(MaxPooling2D(pool_size= (4,4), name="MaxPooling2D_2"))
model.add(Dropout(0.5, name="Dropout_2")) # leave random number of weights untouched for a training cycle, here 50%

#conv block 3
model.add(Conv2D(128, kernel_size=(4,4),activation="relu", kernel_constraint=max_norm(3.)))
#model.add(Conv2D(128, kernel_size=(4,4),activation="relu", kernel_constraint=max_norm(3.)))
model.add(MaxPooling2D(pool_size= (4,4)))
model.add(Dropout(0.5)) # leave random number of weights untouched for a training cycle, here 50%

# Identification
model.add(Flatten())
model.add(Dense(128, activation="relu", name="features"))

# last layer to categories
model.add(Dense(nb_classes, activation="softmax"))# take the output on the 10 neurons and make a propability distribution


model.compile(loss="categorical_crossentropy",
             optimizer="Adam",
             metrics=["accuracy"]) #just numbers to evaluate the training process like number of images correcly categorized

model.summary()

(128, 128, 3)
Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv2D_1 (Conv2D)            (None, 127, 127, 32)      416       
_________________________________________________________________
MaxPooling2D_1 (MaxPooling2D (None, 63, 63, 32)        0         
_________________________________________________________________
Conv2D_2 (Conv2D)            (None, 60, 60, 64)        32832     
_________________________________________________________________
MaxPooling2D_2 (MaxPooling2D (None, 15, 15, 64)        0         
_________________________________________________________________
Dropout_2 (Dropout)          (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 12, 12, 128)       131200    
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 3, 

# Save Directory and Name

In [10]:
my_epochs = 30
steps=  50

In [11]:
# Naming based on the time (to avoid overwriting models) and a buzzword 
import datetime
NN_DIR = "Ausarbeitung/NN/keras/Network/kernel_size/OneSort/"
DATE = datetime.datetime.now().strftime("D_%d_H_%H_M_%M")
ADD_NAME = "_" + str(model.count_params()) +"param_" + str(batchsize) + "batch_" + str(my_epochs) + "epoch_AUGset1_combined_kernel_small" + str(nb_train_samples) + "smpl"
LOGDIR = NN_DIR + DATE + ADD_NAME

print(LOGDIR)

Ausarbeitung/NN/keras/Network/kernel_size/OneSort/D_17_H_16_M_28_58725param_32batch_30epoch_AUGset1_combined_kernel_small1773smpl


# Training NN

In [12]:
# To View the acurracy and vaL_acurracy in a graph afterwards
my_tensorboard = TensorBoard(log_dir = LOGDIR,
                            histogram_freq = 1,
                            write_graph = True,
                            write_images = True)

In [13]:
history = model.fit(train_generator,
                    epochs=my_epochs,
                    steps_per_epoch=steps,
                    callbacks=[my_tensorboard], #, tf.keras.callbacks.EarlyStopping(patience=8, verbose=True, restore_best_weights=True)],
                    verbose=1,
                    validation_data=val_generator,
                    validation_steps=steps,
                    workers=1,
                    use_multiprocessing=False)

Epoch 1/30
50/50 [==============================] - 33s 658ms/step - loss: 1.6106 - accuracy: 0.2188 - val_loss: 1.6080 - val_accuracy: 0.2000
Epoch 2/30
50/50 [==============================] - 30s 594ms/step - loss: 1.5682 - accuracy: 0.2929 - val_loss: 1.5438 - val_accuracy: 0.3200
Epoch 3/30
50/50 [==============================] - 30s 600ms/step - loss: 1.5242 - accuracy: 0.3042 - val_loss: 1.5341 - val_accuracy: 0.2600
Epoch 4/30
50/50 [==============================] - 31s 611ms/step - loss: 1.4710 - accuracy: 0.3346 - val_loss: 1.3609 - val_accuracy: 0.6200
Epoch 5/30
50/50 [==============================] - 29s 590ms/step - loss: 1.4029 - accuracy: 0.3852 - val_loss: 1.2476 - val_accuracy: 0.5600
Epoch 6/30
50/50 [==============================] - 29s 583ms/step - loss: 1.3425 - accuracy: 0.4326 - val_loss: 1.1962 - val_accuracy: 0.7400
Epoch 7/30
50/50 [==============================] - 32s 650ms/step - loss: 1.2828 - accuracy: 0.4541 - val_loss: 1.1340 - val_accuracy: 0.5800

In [14]:
model.save(LOGDIR + ".h5")

In [15]:
from RunTB import run_tensorboard
run_tensorboard(LOGDIR)

In [16]:
LOGDIR = "Ausarbeitung/NN/D_25_H_22_M_02_S_34_front_312kp_32batch_60epoch_combined/"